In [1]:
import os
import sys
import warnings
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston, load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from autofeat import AutoFeatRegressor

# %matplotlib inline
# %load_ext autoreload
# %autoreload 2

#non-linear transformations are user-selectable? 


#website with good autofeat explanation: 
#https://analyticsindiamag.com/guide-to-automatic-feature-engineering-using-autofeat/
# this shows other good feature engineering tools https://neptune.ai/blog/feature-engineering-tools

In [2]:


        # df = pd.read_csv("AdjustedData.csv",header=0) #names=["HoF","E_Hull","Length(x)","r-(M)","r-(X)","r-(T)","IP(M)","IP(X)","IP(T)","EN(M)","EN(X)","EN(T)","EA(M)","EA(X)","EA(T)","Workfunction"])
        # df.dropna()
        # X=df[["HoF","E_Hull","Length(x)","r-(M)","r-(X)","r-(T)","IP(M)","IP(X)","IP(T)","EN(M)","EN(X)","EN(T)","EA(M)","EA(X)","EA(T)"]]
        # X=X.to_numpy()
        # y=df["Workfunction"].to_numpy()   


In [22]:
datasets = ["C2DB"]
def load_regression_dataset(name):
    # load one of the datasets as X and y (and possibly units)
    units = {}
    if name == "C2DB":
        units = {"x001": "eV", "x002": "eV", "x003": "pm", "x004": "pm","x005": "pm", "x006": "pm", "x007": "eV", "x008": "eV","x009": "eV", "x013": "eV", "x014": "eV", "x015": "eV"}
        df = pd.read_csv("AdjustedData2.csv")
        drop_A=df.index[df["E_Hull"] == 0].tolist() #dropping rows that have 0 - only applies to E_Hull. 11 rows. 
        df=df.drop(df.index[drop_A])
        X =df.iloc[:, 1:].to_numpy()
        y=df["Workfunction"].to_numpy()   
#don't need to care about the unit for y?  
    else:
        raise RuntimeError("Unknown dataset %r" % name)
    return np.array(X, dtype=float), np.array(y, dtype=float), units

#here, need to add in the unit details. 
#ev/atom is not recognized, so i will just input as ev. 

In [23]:
load_regression_dataset("C2DB")

(array([[-1.34595169e+00,  2.42879982e-01,  3.30400000e+02,
          8.29000000e+01,  5.96000000e+01,  4.14000000e+01,
          6.82500000e+00,  1.12603000e+01,  1.30170000e+01,
          1.30000000e+00,  2.55000000e+00,  2.82000000e+00,
          1.78000000e-01,  1.26200000e+00,  1.82700000e+00],
        [-1.19995430e+00,  4.52743884e-01,  3.35000000e+02,
          5.39000000e+01,  5.96000000e+01,  4.14000000e+01,
          6.56140000e+00,  1.12603000e+01,  1.30170000e+01,
          1.36000000e+00,  2.55000000e+00,  2.82000000e+00,
          1.89000000e-01,  1.26200000e+00,  1.82700000e+00],
        [-3.17344146e-01,  4.73692689e-01,  2.90600000e+02,
          7.46000000e+01,  5.96000000e+01,  4.14000000e+01,
          7.98000000e+00,  1.12603000e+01,  1.30170000e+01,
          2.36000000e+00,  2.55000000e+00,  2.82000000e+00,
          8.16000000e-01,  1.26200000e+00,  1.82700000e+00],
        [-5.94738539e-01,  3.27053005e-01,  2.85000000e+02,
          7.02000000e+01,  4.88000000

In [24]:
def test_model(dataset, model, param_grid):
    # load data
    X, y, _ = load_regression_dataset(dataset)
    # split in training and test parts
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
    if model.__class__.__name__ == "SVR":
        sscaler = StandardScaler()
        X_train = sscaler.fit_transform(X_train)
        X_test = sscaler.transform(X_test) #value standardization z = (x - u) / s 
    # train model on train split incl cross-validation for parameter selection
    gsmodel = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5) #GridSearchCV is a cross-validation method 
    gsmodel.fit(X_train, y_train)
    print("best params:", gsmodel.best_params_)
    print("best score:", gsmodel.best_score_)
    print("MSE on training data:", mean_squared_error(y_train, gsmodel.predict(X_train)))
    print("MSE on test data:", mean_squared_error(y_test, gsmodel.predict(X_test)))
    print("R^2 on training data:", r2_score(y_train, gsmodel.predict(X_train)))
    print("R^2 on test data:", r2_score(y_test, gsmodel.predict(X_test)))
    return gsmodel.best_estimator_

def test_autofeat(dataset, feateng_steps=2):
    # load data
    X, y, units = load_regression_dataset(dataset)
    # split in training and test parts
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)
    # run autofeat
    afreg = AutoFeatRegressor(verbose=2, feateng_steps=feateng_steps, units=units,transformations=("exp", "log", "abs", "sqrt", "^2", "^3", "1+", "1-", "sin", "cos", "exp-", "2^"))
    # fit autofeat on less data, otherwise ridge reg model with xval will overfit on new features
    X_train_tr = afreg.fit_transform(X_train, y_train)
    X_test_tr = afreg.transform(X_test)
    print("autofeat new features:", len(afreg.new_feat_cols_))
    print("autofeat MSE on training data:", mean_squared_error(y_train, afreg.predict(X_train_tr)))
    print("autofeat MSE on test data:", mean_squared_error(y_test, afreg.predict(X_test_tr)))
    print("autofeat R^2 on training data:", r2_score(y_train, afreg.predict(X_train_tr)))
    print("autofeat R^2 on test data:", r2_score(y_test, afreg.predict(X_test_tr)))
    # train rreg on transformed train split incl cross-validation for parameter selection #ridge regression. 
    print("# Ridge Regression")
    rreg = Ridge()
    param_grid = {"alpha": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1., 2.5, 5., 10., 25., 50., 100., 250., 500., 1000., 2500., 5000., 10000.]}
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        gsmodel = GridSearchCV(rreg, param_grid, scoring='neg_mean_squared_error', cv=5)
        gsmodel.fit(X_train_tr, y_train)
    print("best params:", gsmodel.best_params_)
    print("best score:", gsmodel.best_score_)
    print("MSE on training data:", mean_squared_error(y_train, gsmodel.predict(X_train_tr)))
    print("MSE on test data:", mean_squared_error(y_test, gsmodel.predict(X_test_tr)))
    print("R^2 on training data:", r2_score(y_train, gsmodel.predict(X_train_tr)))
    print("R^2 on test data:", r2_score(y_test, gsmodel.predict(X_test_tr)))
    print("# Random Forest")
    rforest = RandomForestRegressor(n_estimators=100, random_state=13)
    param_grid = {"min_samples_leaf": [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2]}
    gsmodel = GridSearchCV(rforest, param_grid, scoring='neg_mean_squared_error', cv=5)
    gsmodel.fit(X_train_tr, y_train)
    print("best params:", gsmodel.best_params_)
    print("best score:", gsmodel.best_score_)
    print("MSE on training data:", mean_squared_error(y_train, gsmodel.predict(X_train_tr)))
    print("MSE on test data:", mean_squared_error(y_test, gsmodel.predict(X_test_tr)))
    print("R^2 on training data:", r2_score(y_train, gsmodel.predict(X_train_tr)))
    print("R^2 on test data:", r2_score(y_test, gsmodel.predict(X_test_tr)))

In [25]:
for dsname in datasets:
    print("####", dsname)
    X, y, _ = load_regression_dataset(dsname)
    print(X.shape)
    
##why inconsistent with excel? the website could have been updated 

#### C2DB
(264, 15)


In [26]:
np.set_printoptions(threshold=sys.maxsize)
X

array([[-1.34595169e+00,  2.42879982e-01,  3.30400000e+02,
         8.29000000e+01,  5.96000000e+01,  4.14000000e+01,
         6.82500000e+00,  1.12603000e+01,  1.30170000e+01,
         1.30000000e+00,  2.55000000e+00,  2.82000000e+00,
         1.78000000e-01,  1.26200000e+00,  1.82700000e+00],
       [-1.19995430e+00,  4.52743884e-01,  3.35000000e+02,
         5.39000000e+01,  5.96000000e+01,  4.14000000e+01,
         6.56140000e+00,  1.12603000e+01,  1.30170000e+01,
         1.36000000e+00,  2.55000000e+00,  2.82000000e+00,
         1.89000000e-01,  1.26200000e+00,  1.82700000e+00],
       [-3.17344146e-01,  4.73692689e-01,  2.90600000e+02,
         7.46000000e+01,  5.96000000e+01,  4.14000000e+01,
         7.98000000e+00,  1.12603000e+01,  1.30170000e+01,
         2.36000000e+00,  2.55000000e+00,  2.82000000e+00,
         8.16000000e-01,  1.26200000e+00,  1.82700000e+00],
       [-5.94738539e-01,  3.27053005e-01,  2.85000000e+02,
         7.02000000e+01,  4.88000000e+01,  4.14000000

In [27]:
X_inf = np.where(np.isinf(X))
print (X_inf)

(array([], dtype=int64), array([], dtype=int64))


In [28]:
# df.isna().sum()
# # how to deal with this missing value? 

# df.isnull().any() #np.isnan(df.any())

# np.isfinite(X.any())
# # SimpleImputer(missing_values='NaN', strategy='median', axis=1) 
# # imp.fit(df)
# # df = imp.fit_transform(df)

# df = df.replace([np.inf, -np.inf], np.nan)
# df
# np.isfinite(X.any())

# np.set_printoptions(threshold=sys.maxsize)
# X

In [29]:
for dsname in datasets:
    print("####", dsname)
    rreg = Ridge()
    params = {"alpha": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1., 2.5, 5., 10., 25., 50., 100., 250., 500., 1000., 2500., 5000., 10000., 25000., 50000., 100000.]}
    rreg = test_model(dsname, rreg, params)
    
    
## is it correct to interpret that, ridge regression (best alpha 1.0) has high MSE, but moderate-low R^2? 

#### C2DB
best params: {'alpha': 0.1}
best score: -1.1264770553147774
MSE on training data: 1.0160990564610142
MSE on test data: 1.1366198053884218
R^2 on training data: 0.5818370782548532
R^2 on test data: 0.4996782491126156


In [30]:
for dsname in datasets:
    print("####", dsname)
    svr = SVR(gamma="scale")
    params = {"C": [1., 10., 25., 50., 100., 250.]}
    svr = test_model(dsname, svr, params)
    
#support vector regression - it performs quite well. but is theMSE gap between training and test data concerning? is it an overfit


#### C2DB
best params: {'C': 50.0}
best score: -0.16683517922401403
MSE on training data: 0.012203339046412217
MSE on test data: 0.21740907479752325
R^2 on training data: 0.9949778676810629
R^2 on test data: 0.9043000232392298


In [31]:
for dsname in datasets:
    print("####", dsname)
    rforest = RandomForestRegressor(n_estimators=100, random_state=13)
    params = {"min_samples_leaf": [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2]}
    rforest = test_model(dsname, rforest, params)

#MSE - mean squared error (for this mode, shown below)
#MAE mean absolute error. (for pranav, five fold cross val. 0.1 on train, 0.25 on test.)

# it does look like it overfit. 

#### C2DB
best params: {'min_samples_leaf': 0.0001}
best score: -0.15513655959907396
MSE on training data: 0.020045694856006702
MSE on test data: 0.395285029071284
R^2 on training data: 0.9917504437425673
R^2 on test data: 0.826001890071825


In [32]:
for dsname in datasets:
    print("####", dsname)
    test_autofeat(dsname, feateng_steps=1)

#i want to be able to see the X dataset values! 

#### C2DB
[AutoFeat] Applying the Pi Theorem
[AutoFeat] Pi Theorem 1:  x002 / x001
[AutoFeat] Pi Theorem 2:  x004 / x003
[AutoFeat] Pi Theorem 3:  x005 / x003
[AutoFeat] Pi Theorem 4:  x006 / x003
[AutoFeat] Pi Theorem 5:  x007 / x001
[AutoFeat] Pi Theorem 6:  x008 / x001
[AutoFeat] Pi Theorem 7:  x009 / x001
[AutoFeat] Pi Theorem 8:  x013 / x001
[AutoFeat] Pi Theorem 9:  x014 / x001
[AutoFeat] The 1 step feature engineering process could generate up to 180 features.
[AutoFeat] With 211 data points this new feature matrix would use about 0.00 gb of space.
[feateng] Step 1: transformation of original features
[feateng] Generated 52 transformed features from 15 original features - done.
[feateng] Generated altogether 55 new features in 1 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a total of 18 additional features
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel]	 9 initial features.
[featsel]	 Spl

In [33]:
for dsname in datasets:
    print("####", dsname)
    test_autofeat(dsname, feateng_steps=2)

#### C2DB
[AutoFeat] Applying the Pi Theorem
[AutoFeat] Pi Theorem 1:  x002 / x001
[AutoFeat] Pi Theorem 2:  x004 / x003
[AutoFeat] Pi Theorem 3:  x005 / x003
[AutoFeat] Pi Theorem 4:  x006 / x003
[AutoFeat] Pi Theorem 5:  x007 / x001
[AutoFeat] Pi Theorem 6:  x008 / x001
[AutoFeat] Pi Theorem 7:  x009 / x001
[AutoFeat] Pi Theorem 8:  x013 / x001
[AutoFeat] Pi Theorem 9:  x014 / x001
[AutoFeat] The 2 step feature engineering process could generate up to 16290 features.
[AutoFeat] With 211 data points this new feature matrix would use about 0.01 gb of space.
[feateng] Step 1: transformation of original features
[feateng] Generated 52 transformed features from 15 original features - done.
[feateng] Step 2: first combination of features
[feateng] Generated 2191 feature combinations from 2211 original feature tuples - done.
[feateng] Generated altogether 2253 new features in 2 steps
[feateng] Removing correlated features, as well as additions at the highest level
[feateng] Generated a tota

/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only suppo

best params: {'min_samples_leaf': 0.0001}
best score: -0.1507355208199151
MSE on training data: 0.02041124207373446
MSE on test data: 0.4162519985725981
R^2 on training data: 0.9916000073341986
R^2 on test data: 0.8167725674417174


/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only suppo

In [34]:
for dsname in datasets:
    print("####", dsname)
    test_autofeat(dsname, feateng_steps=3)

    #what does x001 mean, and what is the final equation? 
    

#### C2DB
[AutoFeat] Applying the Pi Theorem
[AutoFeat] Pi Theorem 1:  x002 / x001
[AutoFeat] Pi Theorem 2:  x004 / x003
[AutoFeat] Pi Theorem 3:  x005 / x003
[AutoFeat] Pi Theorem 4:  x006 / x003
[AutoFeat] Pi Theorem 5:  x007 / x001
[AutoFeat] Pi Theorem 6:  x008 / x001
[AutoFeat] Pi Theorem 7:  x009 / x001
[AutoFeat] Pi Theorem 8:  x013 / x001
[AutoFeat] Pi Theorem 9:  x014 / x001
[AutoFeat] The 3 step feature engineering process could generate up to 725130 features.
[AutoFeat] With 211 data points this new feature matrix would use about 0.61 gb of space.
[feateng] Step 1: transformation of original features
[feateng] Generated 52 transformed features from 15 original features - done.
[feateng] Step 2: first combination of features
[feateng] Generated 3869 feature combinations from 2211 original feature tuples - done.
[feateng] Step 3: transformation of new features
[feateng] Generated 20168 transformed features from 3869 original features - done.
[feateng] Generated altogether 2515

/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only suppo

best params: {'min_samples_leaf': 0.0001}
best score: -0.1722078068233132
MSE on training data: 0.023110931879313148
MSE on test data: 0.3080801404967798
R^2 on training data: 0.9904889835912594
R^2 on test data: 0.8643880789545932


/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'str_']. An error will be raised in 1.2.
  warnings.warn(
/Users/seojiwoo/Desktop/URECA_May/autofeat/autofeat/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only suppo

In [ ]:
#how to split test and train? either test on the new termination, or just shuffle everything together. 
#how do i update the different units? 

#summary of the performance: 

#for autofeat (4 feature engineering)
#MSE on training data: 0.02159085270575962
#MSE on test data: 0.24196610436647903
#is this indication of overfitting? difference scale 11. 